In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
class Perceptron():
    def __init__(self,train=True,theta=.5,alpha=1e-1):
        self.weights=[]
        self.theta=theta
        self.alpha=alpha
    def expand(self,x):
        x_dim=x.shape[0]
        while(x_dim>len(self.weights)): 
            self.weights+=[.5]
        return x,x_dim
    def __call__(self,x,y=None):
        x,x_dim=self.expand(x)
        if np.dot(np.array(self.weights[0:x_dim]),x)>=self.theta: pred=1
        else: pred=0
        if y==1 and pred==0:
            for i in range(x_dim):
                if x[i]==1: 
                    self.weights[i]+=self.alpha
        if y==0 and pred==1:
            for i in range(x_dim):
                if x[i]==1: self.weights[i]-=self.alpha
        return pred

In [ ]:
class Winnow():
    def __init__(self,train=True,theta=.5,alpha=2.0):
        self.weights=[]
        self.train=train
        self.theta=theta
        self.alpha=alpha
    def expand(self,x):
        x_dim=x.shape[0]
        while(x_dim>len(self.weights)): 
            self.weights+=[.5]
        return x,x_dim
    def __call__(self,x,y=None):
        x,x_dim=self.expand(x)
        if np.dot(np.array(self.weights[0:x_dim]),x)>=self.theta: pred=1
        else: pred=0
        if y==1 and pred==0:
            for i in range(x_dim):
                if x[i]==1: 
                    self.weights[i]*=self.alpha
        if y==0 and pred==1:
            for i in range(x_dim):
                if x[i]==1: self.weights[i]/=self.alpha
        return pred

In [ ]:
DATA_PATH = "../data/breast_cancer_dataset.csv"
pd_data = pd.read_csv(DATA_PATH, sep=",")
data=pd_data.values
pd_training_data = pd_data.sample(frac=.5, random_state=99)
np_training_data = pd_training_data.values[:,1:]
pd_testing_data = pd_data.drop(pd_training_data.index)
np_testing_data = pd_testing_data.values[1:,1:]

In [ ]:
#Random data
data=np.random.choice([1,0],size=(500,50))
data=np.concatenate((data,data[:,-1:]),axis=1)
np_training_data=data[0:250,:]
np_testing_data=data[250:,:]

In [ ]:
net=Winnow()

In [ ]:
net=Perceptron(alpha=1)

In [ ]:
n_epochs=1

In [ ]:
def test_accuracy(net,data):
    m,n=0,0
    for s in np_testing_data:
        x,y=s[0:-1],s[-1]
        pred=net(x)
        if pred==y: n+=1
        m+=1
    return n/m
    # print(f'{n} of {m} correct; accuracy {n/m}')

In [ ]:
m,n=0,0
accL=[]
for e in range(n_epochs):
    for s in np_training_data:
        x,y=s[0:-1],s[-1]
        pred=net(x,y)
        accL+=[test_accuracy(net,np_testing_data)]
        if pred==y: n+=1
        m+=1
print(net.weights)
print(f'{n} of {m} correct; accuracy {n/m}')

In [ ]:
test_accuracy(net,np_testing_data)

In [ ]:
import plotly.express as px

In [ ]:
px.line(accL).show()

In [ ]:
px.line(accL).show()

In [ ]:
#Random data concept drift
#Target concept changes R times, each time a random column is the concept
R=5
all_data=np.ones((1,51))
for r in range(R):
    c=np.random.choice([i for i in range(50)])
    data=np.random.choice([1,0],size=(100,50))
    data=np.concatenate((data,data[:,c:c+1]),axis=1)
    all_data=np.concatenate((all_data,data))
    # np_training_data=data[0:250,:]
    # np_testing_data=data[250:,:]
np_training_data=all_data

In [ ]:
net=Winnow()

In [ ]:
net=Perceptron(alpha=.1)

In [ ]:
m,n=0,0
accL=[]
for e in range(n_epochs):
    for s in np_training_data:
        x,y=s[0:-1],s[-1]
        pred=net(x,y)
        if pred==y: n+=1
        m+=1
        accL+=[n/m]
print(net.weights)
print(f'{n} of {m} correct; accuracy {n/m}')

In [ ]:
px.line(accL).show()

In [ ]:
px.line(accL).show()